# Surgical Trends Data Cleaning Pipeline (2022–2025)

## Purpose
This notebook builds a reproducible data-cleaning pipeline for hospital theatre records (2022–2025).  
The goal is to convert raw multi-year Excel files into a single standardized master dataset that is ready for analysis and dashboarding.

This notebook focuses on:
- Standardizing column names and categorical values
- Parsing dates and creating time features
- Cleaning and standardizing age entries into numeric years
- Creating clinically meaningful age groups
- Handling missing anesthesia values using clinically informed rules
- Producing and exporting a final master dataset for the 

## Inputs

Raw Excel files:
- 2022.xlsx
- 2023.xlsx
- 2024.xlsx
- 2025.xlsx

## Expected Core Columns (after standardization)

- MONTH (date of surgery)
- AGE (raw entry as recorded)
- SEX
- TYPE_OF_SURGERY (EL / EM)
- SPECIALTY
- DIAGNOSIS
- OPERATION## Why a Pipeline?

Hospital operational data often contains inconsistencies across years due to:
- Different data entry practices
- Variations in column naming
- Mixed age formats (years, months, days, year of birth)
- Missing or incomplete anesthesia entries

Instead of manually cleaning each file, this notebook defines reusable functions to ensure:
- Consistency across years
- Reproducibility (same input → same output)
- Reduced human error
- Easier scaling when new year
## Data Cleaning Strategy

The pipeline is applied to each year’s raw dataset and includes the following steps:

1. **Standardize Column Names**
   - Strip extra spaces, uppercase, replace spaces with underscores

2. **Parse MONTH as Datetime**
   - Ensure MONTH is treated as a date column for trend analysis

3. **Clean Categorical Values**
   - Strip whitespace and uppercase values for consistency

4. **Standardize TYPE_OF_SURGERY**
   - Harmonize common variants into EL (elective) and EM (emergency)

5. **Create Time Features**
   - YEAR, MONTH_NAME, YEAR_MONTH (for time-series analysis)

6. **Age Standardization**
   - Convert mixed AGE entries into numeric **AGE_YEARS**
   - Create **AGE_GROUP** for clinically meaningful comparisons

7. **Anesthesia Handling**
   - For missing anesthesia:
     - OBS/ORTHO → SA
     - Others → GA
   - Flag imputed values u

## Feature Engineering Notes

### AGE_YEARS
AGE was standardized into numeric years using the following rules:
- Numeric ages in years are kept as-is (including decimals)
- Entries that appear to be year-of-birth (e.g., 1974) are converted using: surgery_year − birth_year
- Clear month/day expressions are converted into years:
  - X months → X/12 years
  - X days → X/365 years
- Invalid or ambiguous entries are set to missing (NaN)

### AGE_GROUP
AGE_YEARS is binned into clinically meaningful categories:
- 0–<1, 1–5, 6–12, 13–18, 19–35, 36–60, 60+

### ANESTHESIA_IMPUTED
ANESTHESIA_IMPUTED indicates whether anesthesia was missing and filled using the clinical imputation rule.
This supports transparency and allows sensitivity checks

 during analysis.
sing **ANESTHESIA_IMPUTED**
s are added
S
- ANESTHESIA
EDA notebook
et.


In [1]:
import pandas as pd
import numpy as np
import re


## Reusable Cleaning Functions

The following functions implement the cleaning steps described above.  
They are designed to be applied consistently across all years (2022–2025).


In [3]:
# ---------- Column name standardization ----------
def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .astype(str)
        .str.strip()
        .str.upper()
        .str.replace(" ", "_", regex=False)
    )
    return df


# ---------- Clean text values ----------
def clean_text_values(df, cols):
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype("string").str.strip().str.upper()
    return df


# ---------- Ensure MONTH is datetime ----------
def ensure_month_datetime(df, month_col="MONTH"):
    df = df.copy()
    df[month_col] = pd.to_datetime(df[month_col], errors="coerce")
    return df


# ---------- Add time features ----------
def add_time_features(df, month_col="MONTH"):
    df = df.copy()
    df["YEAR"] = df[month_col].dt.year
    df["MONTH_NAME"] = df[month_col].dt.month_name()
    df["YEAR_MONTH"] = df[month_col].dt.to_period("M").astype(str)
    return df


# ---------- Standardize surgery type ----------
def standardize_surgery_type(df, col="TYPE_OF_SURGERY"):
    df = df.copy()
    mapping = {
        "ELECTIVE": "EL",
        "EL": "EL",
        "EMERGENCY": "EM",
        "EM": "EM"
    }
    df[col] = df[col].replace(mapping)
    return df


# ---------- Convert AGE to years ----------
def convert_age_to_years(age, surgery_date):
    if pd.isna(age) or pd.isna(surgery_date):
        return np.nan

    age_str = str(age).strip().lower()
    surgery_year = surgery_date.year

    # Try numeric first
    try:
        age_val = float(age_str)

        if 0 <= age_val <= 120:
            return age_val

        if age_val.is_integer():
            y = int(age_val)
            if 1900 <= y <= surgery_year:
                return surgery_year - y

        return np.nan
    except ValueError:
        pass

    # Months
    if "month" in age_str:
        nums = re.findall(r"\d+", age_str)
        if nums:
            return int(nums[0]) / 12

    # Days
    if "day" in age_str:
        nums = re.findall(r"\d+", age_str)
        if nums:
            return int(nums[0]) / 365

    return np.nan


# ---------- Add age features ----------
def add_age_features(df, age_col="AGE", month_col="MONTH"):
    df = df.copy()

    df["AGE_NUM"] = pd.to_numeric(df[age_col], errors="coerce")
    df["AGE_YEARS"] = df.apply(
        lambda r: convert_age_to_years(r[age_col], r[month_col]), axis=1
    )

    bins = [0, 1, 5, 12, 18, 35, 60, 200]
    labels = ["0–<1", "1–5", "6–12", "13–18", "19–35", "36–60", "60+"]

    df["AGE_GROUP"] = pd.cut(df["AGE_YEARS"], bins=bins, labels=labels, right=False)
    return df


# ---------- Impute anesthesia ----------
def impute_anesthesia(df, anesthesia_col="ANESTHESIA", specialty_col="SPECIALTY"):
    df = df.copy()

    df["ANESTHESIA_IMPUTED"] = df[anesthesia_col].isna()

    df.loc[
        (df[anesthesia_col].isna()) &
        (df[specialty_col].isin(["OBS", "OBSTETRICS", "ORTHO", "ORTHOPAEDICS"])),
        anesthesia_col
    ] = "SA"

    df.loc[df[anesthesia_col].isna(), anesthesia_col] = "GA"

    return df


# ---------- Full pipeline ----------
def process_one_year(file_path):
    df = pd.read_excel(file_path)

    df = standardize_columns(df)
    df = ensure_month_datetime(df)

    cat_cols = ["SEX", "TYPE_OF_SURGERY", "SPECIALTY", "DIAGNOSIS", "OPERATIONS", "ANESTHESIA"]
    df = clean_text_values(df, cat_cols)

    df = standardize_surgery_type(df)
    df = add_time_features(df)
    df = add_age_features(df)
    df = impute_anesthesia(df)

    return df


In [4]:
process_one_year


<function __main__.process_one_year(file_path)>

## Pipeline Test (2022)

Before processing all years, the pipeline is tested on the raw 2022 dataset to confirm:
- Correct column standardization
- Correct time feature generation
- Correct AGE_YEARS and AGE_GROUP creation
- Correct anesthesia imputation and flagging

Once validated, the same pipeline is applied to the remaining years.


In [5]:
df_2022_final = process_one_year("2022.xlsx")
df_2022_final.head()


,MONTH,AGE,SEX,TYPE_OF_SURGERY,SPECIALTY,DIAGNOSIS,OPERATIONS,ANESTHESIA,YEAR,MONTH_NAME,YEAR_MONTH,AGE_NUM,AGE_YEARS,AGE_GROUP,ANESTHESIA_IMPUTED
0,2022-01-01,27,MALE,EL,OPHTHA,CORNEAL LACERATION,CORNEAL REPAIR,LA,2022,January,2022-01,27.0,27.0,19–35,False
1,2022-01-01,9,MALE,EL,OPHTHA,CORNEAL PERFORATION,CORNEAL REPAIR,GA,2022,January,2022-01,9.0,9.0,6–12,False
2,2022-01-01,32,FEMALE,EM,GYNAE,RIGHT ECTOPIC PREGNANCY,EXPLORATORY LAPAROTOMY/SALPINGECTOMY,GA,2022,January,2022-01,32.0,32.0,19–35,False
3,2022-01-01,43,MALE,EM,GEN. SURGERY,MULTIPLE FACIAL LACERATION,DEBRIDEMENT/STITCHING,GA,2022,January,2022-01,43.0,43.0,36–60,False
4,2022-01-01,22,MALE,EM,GEN. SURGERY,COMPLICATED APPENDICITS,APPENDICITS,GA,2022,January,2022-01,22.0,22.0,19–35,False


## Processing 2022–2025 and Creating the Master Dataset

After validating the pipeline on 2022, the pipeline is applied to:
- 2023
- 2024
- 2025

The cleaned yearly datasets are concatenated into a single master dataset for analysis.


In [7]:
df_2023_final = process_one_year("2023.xlsx")
df_2024_final = process_one_year("2024.xlsx")
df_2025_final = process_one_year("2025.xlsx")


In [8]:
df_2023_final.head()
df_2024_final.head()
df_2025_final.head()


,MONTH,AGE,SEX,TYPE_OF_SURGERY,SPECIALTY,DIAGNOSIS,OPERATIONS,ANESTHESIA,YEAR,MONTH_NAME,YEAR_MONTH,AGE_NUM,AGE_YEARS,AGE_GROUP,ANESTHESIA_IMPUTED
0,2025-01-01,20,FEMALE,EM,OBSTETRICS,1PREVIOUS SCAR POST TERM PREGNANCY,EMERGENCY CAESAREAN SECTION,SPINAL,2025.0,January,2025-01,20.0,20.0,19–35,False
1,2025-01-01,39,FEMALE,EM,OBSTETRICS,BREECH PRESENTATION IN ACTIVE PHASE OF LABOR,EMERGENCY CAESAREAN SECTION,SPINAL,2025.0,January,2025-01,39.0,39.0,36–60,False
2,2025-01-01,28,FEMALE,EM,OBSTETRICS,1PREVIOUS SCAR DECLINED TOLAC,EMERGENCY CAESAREAN SECTION,SPINAL,2025.0,January,2025-01,28.0,28.0,19–35,False
3,2025-01-01,21,FEMALE,EM,OBSTETRICS,NON REASSURING FETAL STATUS,EMERGENCY CAESAREAN SECTION,SPINAL,2025.0,January,2025-01,21.0,21.0,19–35,False
4,2025-01-01,9,MALE,EM,ORTHOPAEDIC,FRACTURE RADIUS/ ULNA,CLOSED REDUCTION PLUS CAST,GENERAL,2025.0,January,2025-01,9.0,9.0,6–12,False


## Output

The final output of this notebook is a single cleaned dataset:

- **SURGICAL_MASTER_2022_2025.xlsx**

This dataset is used in the next notebook:
- `03_EDA_and_Insights_2022_2025.ipynb`


In [9]:
master_df = pd.concat(
    [df_2022_final, df_2023_final, df_2024_final, df_2025_final],
    ignore_index=True
)

master_df.shape


(62651, 17)

In [10]:
master_df.to_excel("SURGICAL_MASTER_2022_2025.xlsx", index=False)
print("✅ Master dataset saved successfully")


✅ Master dataset saved successfully


## Data Quality Notes and Limitations

- Theatre records may contain data entry errors (e.g., age entered as ID/name/year of birth).
- Diagnosis and operation text are not standardized with formal coding systems (e.g., ICD/OPS), so text normalization may be required later for deeper clinical grouping.
- Anesthesia imputation uses clinical assumptions for missing entries and should be interpreted accordingly.
